In [ ]:
# In Google Colab
!git clone https://github.com/aswinpradeep/malayalam-asr-datasets.git
%cd malayalam-asr-datasets
!ls -la

In [ ]:
# Check if there are any CSV or metadata files
!find /content/malayalam-asr-datasets/ -name "*.csv" -o -name "*.json" -o -name "*.txt" | head -10

In [ ]:
# Look at the README to understand the dataset structure
!cat /content/malayalam-asr-datasets/README.md

In [ ]:
import os
import librosa
import pandas as pd
from pathlib import Path

def load_malayalam_dataset(dataset_path):
    """Load Malayalam ASR dataset from the specified path"""
    audio_files = []
    transcriptions = []

    # Look for common ASR dataset structures
    audio_dir = os.path.join(dataset_path, "audio")  # or "wav", "clips"
    transcript_file = os.path.join(dataset_path, "transcripts.txt")  # or "transcript.csv"

    # Check if transcript file exists
    if os.path.exists(transcript_file):
        # Load transcriptions from file
        with open(transcript_file, 'r', encoding='utf-8') as f:
            lines = f.readlines()

        for line in lines:
            # Assuming format: filename|transcription
            parts = line.strip().split('|')
            if len(parts) >= 2:
                filename = parts[0]
                transcription = parts[1]

                audio_path = os.path.join(audio_dir, filename)
                if os.path.exists(audio_path):
                    audio_files.append(audio_path)
                    transcriptions.append(transcription)

    # Alternative: Look for CSV file
    elif os.path.exists(os.path.join(dataset_path, "metadata.csv")):
        df = pd.read_csv(os.path.join(dataset_path, "metadata.csv"))
        for _, row in df.iterrows():
            audio_path = os.path.join(audio_dir, row['filename'])  # adjust column name
            if os.path.exists(audio_path):
                audio_files.append(audio_path)
                transcriptions.append(row['transcription'])  # adjust column name

    # Alternative: Scan directory structure
    else:
        print("No transcript file found. Scanning directory structure...")
        for audio_file in Path(dataset_path).rglob("*.wav"):  # or *.mp3, *.flac
            audio_files.append(str(audio_file))
            # You'd need transcription files alongside audio files
            transcript_file = audio_file.with_suffix('.txt')
            if transcript_file.exists():
                with open(transcript_file, 'r', encoding='utf-8') as f:
                    transcriptions.append(f.read().strip())

    print(f"Found {len(audio_files)} audio files with transcriptions")
    return audio_files, transcriptions

# Now load the dataset
dataset_path = "//content/malayalam-asr-datasets/NISP_MALAYALAM_CLEANED"
audio_files, transcriptions = load_malayalam_dataset(dataset_path)

print(f"Loaded {len(audio_files)} audio files")
if transcriptions:
    print(f"Sample transcription: {transcriptions[0]}")

In [ ]:
# Import required modules
import re
from typing import Dict, List, Any

def preprocess_malayalam_text(text: str) -> str:
    """Preprocess Malayalam text for ASR training"""
    if not text:
        return ""

    # Convert to lowercase
    text = text.lower()

    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    # Add more Malayalam-specific preprocessing as needed
    # Remove punctuation except for essential ones
    text = re.sub(r'[^\u0d00-\u0d7f\s]', '', text)  # Keep only Malayalam characters and spaces

    return text

def create_vocabulary(transcriptions: List[str]) -> Dict[str, int]:
    """Create vocabulary from transcriptions"""
    vocab = set()
    for text in transcriptions:
        if text:
            vocab.update(text)

    # Create vocab dict with special tokens
    vocab_dict = {}
    vocab_dict["<pad>"] = 0
    vocab_dict["<s>"] = 1
    vocab_dict["</s>"] = 2
    vocab_dict["<unk>"] = 3
    vocab_dict["|"] = 4  # word separator

    # Add all characters
    for i, char in enumerate(sorted(vocab)):
        vocab_dict[char] = i + 5

    return vocab_dict

print("Functions defined successfully!")

In [ ]:
# Continue with text preprocessing
cleaned_transcriptions = [preprocess_malayalam_text(text) for text in transcriptions]

# Create vocabulary
vocab_dict = create_vocabulary(cleaned_transcriptions)
print(f"Vocabulary size: {len(vocab_dict)}")

# Show some vocabulary examples
print("Sample vocabulary entries:")
for i, (char, idx) in enumerate(list(vocab_dict.items())[:20]):
    print(f"'{char}': {idx}")

print(f"\nSample original text: {transcriptions[0]}")
print(f"Sample cleaned text: {cleaned_transcriptions[0]}")

In [ ]:
import json
import os
from transformers import (
    Wav2Vec2CTCTokenizer,
    Wav2Vec2FeatureExtractor,
    Wav2Vec2Processor,
    Wav2Vec2ForCTC,
    TrainingArguments,
    Trainer
)
from datasets import Dataset, Audio

# Save vocabulary
with open("/content/vocab.json", "w", encoding="utf-8") as f:
    json.dump(vocab_dict, f, ensure_ascii=False, indent=2)

# Create tokenizer
tokenizer = Wav2Vec2CTCTokenizer(
    vocab_file="/content/vocab.json",
    unk_token="<unk>",
    pad_token="<pad>",
    word_delimiter_token="|"
)

# Create feature extractor
feature_extractor = Wav2Vec2FeatureExtractor(
    feature_size=1,
    sampling_rate=16000,
    padding_value=0.0,
    do_normalize=True,
    return_attention_mask=True
)

# Create processor
processor = Wav2Vec2Processor(
    feature_extractor=feature_extractor,
    tokenizer=tokenizer
)

print("Tokenizer and processor created successfully!")
print(f"Tokenizer vocab size: {len(tokenizer)}")

In [ ]:
import librosa
import numpy as np
from pathlib import Path
from dataclasses import dataclass
import torch

def load_audio(audio_path: str, target_sr: int = 16000):
    """Load and resample audio file"""
    try:
        audio, sr = librosa.load(audio_path, sr=target_sr)
        return audio
    except Exception as e:
        print(f"Error loading {audio_path}: {e}")
        return np.array([])

print("All required functions imported!")

In [ ]:
# Dataset preparation
def prepare_dataset(audio_files: List[str], transcriptions: List[str]):
    """Prepare dataset for training"""
    dataset_dict = {
        "audio": [],
        "text": []
    }

    print("Loading audio files...")
    successful_loads = 0

    for i, (audio_path, text) in enumerate(zip(audio_files, transcriptions)):
        if i % 100 == 0:
            print(f"Processing {i}/{len(audio_files)} files...")

        if os.path.exists(audio_path) and text:
            audio = load_audio(audio_path)
            if len(audio) > 0:
                dataset_dict["audio"].append(audio)
                dataset_dict["text"].append(text)
                successful_loads += 1

    print(f"Successfully loaded {successful_loads} audio files")
    return Dataset.from_dict(dataset_dict)

# Create dataset
dataset = prepare_dataset(audio_files, cleaned_transcriptions)
print(f"Dataset size: {len(dataset)}")

# Split dataset
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_dataset = dataset.select(range(train_size))
val_dataset = dataset.select(range(train_size, len(dataset)))

print(f"Train size: {len(train_dataset)}, Validation size: {len(val_dataset)}")

In [ ]:
# Data preprocessing function
def prepare_dataset_for_training(batch):
    """Prepare batch for training"""
    # Process audio
    audio = batch["audio"]
    batch["input_values"] = processor(
        audio,
        sampling_rate=16000,
        return_tensors="pt",
        padding=True
    ).input_values[0]

    # Process text
    with processor.as_target_processor():
        batch["labels"] = processor(batch["text"]).input_ids

    return batch

print("Data preprocessing function ready!")

In [ ]:
# Apply preprocessing to datasets
print("Preprocessing training dataset...")
train_dataset = train_dataset.map(
    prepare_dataset_for_training,
    remove_columns=train_dataset.column_names
)

print("Preprocessing validation dataset...")
val_dataset = val_dataset.map(
    prepare_dataset_for_training,
    remove_columns=val_dataset.column_names
)

print("Dataset preprocessing completed!")

In [ ]:
# Data collator
@dataclass
class DataCollatorCTCWithPadding:
    processor: Wav2Vec2Processor
    padding: bool = True

    def __call__(self, features: List[Dict[str, Any]]) -> Dict[str, torch.Tensor]:
        # Split inputs and labels
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )

        labels_batch = self.processor.pad(
            labels=label_features,
            padding=self.padding,
            return_tensors="pt",
        )

        # Replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(
            labels_batch.attention_mask.ne(1), -100
        )

        batch["labels"] = labels
        return batch

# Create data collator
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

print("Data collator created!")

In [ ]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load pre-trained Wav2Vec2 model
print("Loading pre-trained Wav2Vec2 model...")
model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53",
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

# Freeze feature extractor
model.freeze_feature_encoder()

# Move to device
model.to(device)

print("Model loaded and ready for training!")

In [ ]:
# All necessary imports
import torch
import numpy as np
from dataclasses import dataclass
from typing import Dict, List, Union
from transformers import TrainingArguments, Trainer
import pandas as pd

@dataclass
class DataCollatorCTCWithPadding:
    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Union[int, None] = None
    max_length_labels: Union[int, None] = None
    pad_to_multiple_of: Union[int, None] = None
    pad_to_multiple_of_labels: Union[int, None] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # Split inputs and labels since they have to be of different lengths and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # Replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

print("DataCollator defined successfully!")

In [ ]:
!pip install jiwer

In [ ]:
import jiwer
from transformers import TrainingArguments, Trainer
import numpy as np

def compute_metrics(pred):
    """Compute Word Error Rate (WER) for evaluation"""
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # Replace -100 with pad token id
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

    # Decode predictions and labels
    pred_str = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    # Compute WER
    try:
        wer = jiwer.wer(label_str, pred_str)
    except:
        # Fallback to character-level accuracy if WER calculation fails
        total_chars = sum(len(label) for label in label_str)
        if total_chars == 0:
            wer = 0.0
        else:
            correct_chars = sum(
                sum(c1 == c2 for c1, c2 in zip(pred, label))
                for pred, label in zip(pred_str, label_str)
            )
            wer = 1.0 - (correct_chars / total_chars)

    print(f"📊 Current WER: {wer:.4f}")
    return {"wer": wer}

print("✅ Evaluation metrics defined!")

In [ ]:
pip install --upgrade transformers

In [ ]:
from transformers import Trainer, TrainingArguments
import torch

# Clear GPU cache before training
torch.cuda.empty_cache()

# Define TrainingArguments with memory-optimized settings
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=1,  # Reduced to 1 to save memory
    per_device_eval_batch_size=2,   # Small eval batch size
    gradient_accumulation_steps=8,  # Simulate larger batch size (effective batch = 1*8=8)
    num_train_epochs=3,
    save_steps=500,
    eval_strategy="steps",
    eval_steps=500,
    logging_steps=100,
    learning_rate=1e-4,
    warmup_steps=100,
    save_total_limit=2,
    push_to_hub=False,
    report_to="none",

    # Memory optimization settings
    dataloader_pin_memory=False,
    gradient_checkpointing=True,  # Trade compute for memory
    fp16=True,  # Use mixed precision to save memory
    remove_unused_columns=False,

    # Additional memory saving options
    dataloader_num_workers=0,  # Reduce CPU memory usage
    load_best_model_at_end=False,  # Don't keep best model in memory
)

# Create trainer with the correct parameter name
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    processing_class=processor,  # Use processing_class instead of tokenizer
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# Additional memory optimization before training
if hasattr(model, 'gradient_checkpointing_enable'):
    model.gradient_checkpointing_enable()

# Start training
try:
    trainer.train()
except torch.cuda.OutOfMemoryError:
    print("Still out of memory. Try the alternative approach below.")

# Alternative: Even more aggressive memory saving
print("\nIf still getting OOM, try these additional steps:")
print("1. Further reduce batch size to 1")
print("2. Increase gradient_accumulation_steps")
print("3. Use CPU offloading (see code below)")

# Ultra-low memory training configuration
training_args_ultra_low = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=16,  # Higher accumulation
    num_train_epochs=3,
    save_steps=1000,
    eval_strategy="epoch",  # Evaluate less frequently
    logging_steps=50,
    learning_rate=1e-4,
    warmup_steps=50,
    save_total_limit=1,  # Keep only 1 checkpoint
    push_to_hub=False,
    report_to="none",

    # Maximum memory optimization
    dataloader_pin_memory=False,
    gradient_checkpointing=True,
    fp16=True,
    remove_unused_columns=False,
    dataloader_num_workers=0,
    load_best_model_at_end=False,
    eval_accumulation_steps=1,  # Process eval in smaller chunks
)

# Uncomment to use ultra-low memory settings:
# trainer_ultra_low = Trainer(
#     model=model,
#     args=training_args_ultra_low,
#     train_dataset=train_dataset,
#     eval_dataset=val_dataset,
#     processing_class=processor,
#     data_collator=data_collator,
#     compute_metrics=compute_metrics,
# )
# trainer_ultra_low.train()

In [ ]:
# Save the final trained model
print("Saving trained model...")
trainer.save_model("./wav2vec2-finetuned-final")
processor.save_pretrained("./wav2vec2-finetuned-final")
print("✅ Model saved successfully!"
)

In [ ]:
# Define the path to save in Google Drive
drive_save_path = "/content/drive/MyDrive/wav2vec2-finetuned-final"

# Save the final trained model to Google Drive
print("Saving trained model to Google Drive...")
trainer.save_model(drive_save_path)
processor.save_pretrained(drive_save_path)
print("✅ Model saved successfully to Google Drive!")

In [ ]:
!apt-get install -y portaudio19-dev


In [ ]:
!pip install sounddevice


In [ ]:
!pip install sounddevice --force-reinstall


In [ ]:
import torch
import numpy as np
import sounddevice as sd
import threading
import queue
import time
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import librosa
from collections import deque
import warnings
import os
warnings.filterwarnings("ignore")

class RealTimeMalayalamTranscriber:
    def __init__(self, model_path, sample_rate=16000, chunk_duration=2.0, overlap=0.5):
        """
        Initialize the real-time transcriber

        Args:
            model_path: Path to your trained model (e.g., "/content/drive/MyDrive/wav2vec2-finetuned-final")
            sample_rate: Audio sample rate (16kHz is standard for Wav2Vec2)
            chunk_duration: Duration of each audio chunk in seconds
            overlap: Overlap between chunks in seconds
        """
        self.model_path = model_path
        self.sample_rate = sample_rate
        self.chunk_duration = chunk_duration
        self.overlap = overlap
        self.chunk_size = int(sample_rate * chunk_duration)
        self.overlap_size = int(sample_rate * overlap)

        # Audio buffer and processing queue
        self.audio_buffer = deque(maxlen=int(sample_rate * 10))  # 10 seconds buffer
        self.audio_queue = queue.Queue()
        self.transcription_queue = queue.Queue()

        # Control flags
        self.is_recording = False
        self.is_processing = False

        # Load model and processor
        print("Loading trained model...")
        self.load_model()
        print("✅ Model loaded successfully!")

    def load_model(self):
        """Load the trained Wav2Vec2 model and processor"""
        try:
            self.processor = Wav2Vec2Processor.from_pretrained(self.model_path)
            self.model = Wav2Vec2ForCTC.from_pretrained(self.model_path)
            self.model.eval()

            # Move to GPU if available
            self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            self.model.to(self.device)
            print(f"Model loaded on: {self.device}")

        except Exception as e:
            print(f"Error loading model: {e}")
            raise

    def audio_callback(self, indata, frames, time, status):
        """Callback function for audio input"""
        if status:
            print(f"Audio input status: {status}")

        # Convert to mono and add to buffer
        audio_data = indata[:, 0] if indata.ndim > 1 else indata
        self.audio_buffer.extend(audio_data)

        # Add to processing queue if we have enough data
        if len(self.audio_buffer) >= self.chunk_size:
            chunk = np.array(list(self.audio_buffer)[-self.chunk_size:])
            if not self.audio_queue.full():
                self.audio_queue.put(chunk)

    def transcribe_chunk(self, audio_chunk):
        """Transcribe a single audio chunk"""
        try:
            # Preprocess audio
            if len(audio_chunk) == 0:
                return ""

            # Normalize audio
            audio_chunk = audio_chunk / np.max(np.abs(audio_chunk)) if np.max(np.abs(audio_chunk)) > 0 else audio_chunk

            # Process with the model
            with torch.no_grad():
                inputs = self.processor(
                    audio_chunk,
                    sampling_rate=self.sample_rate,
                    return_tensors="pt",
                    padding=True
                )

                inputs = inputs.to(self.device)
                logits = self.model(inputs.input_values).logits

                # Decode the transcription
                predicted_ids = torch.argmax(logits, dim=-1)
                transcription = self.processor.batch_decode(predicted_ids)[0]

                return transcription.strip()

        except Exception as e:
            print(f"Transcription error: {e}")
            return ""

    def processing_worker(self):
        """Worker thread for processing audio chunks"""
        while self.is_processing:
            try:
                # Get audio chunk with timeout
                audio_chunk = self.audio_queue.get(timeout=1.0)

                # Transcribe the chunk
                transcription = self.transcribe_chunk(audio_chunk)

                if transcription and transcription.strip():
                    timestamp = time.strftime("%H:%M:%S")
                    self.transcription_queue.put((timestamp, transcription))

                self.audio_queue.task_done()

            except queue.Empty:
                continue
            except Exception as e:
                print(f"Processing error: {e}")

    def display_worker(self):
        """Worker thread for displaying transcriptions"""
        print("\n" + "="*60)
        print("🎤 REAL-TIME MALAYALAM TRANSCRIPTION")
        print("="*60)
        print("Press Ctrl+C to stop\n")

        while self.is_processing:
            try:
                timestamp, transcription = self.transcription_queue.get(timeout=1.0)
                print(f"[{timestamp}] {transcription}")
                self.transcription_queue.task_done()

            except queue.Empty:
                continue
            except Exception as e:
                print(f"Display error: {e}")

    def start_transcription(self):
        """Start real-time transcription"""
        try:
            print("Starting real-time transcription...")
            print("Make sure your microphone is working!")

            # Set flags
            self.is_recording = True
            self.is_processing = True

            # Start worker threads
            processing_thread = threading.Thread(target=self.processing_worker, daemon=True)
            display_thread = threading.Thread(target=self.display_worker, daemon=True)

            processing_thread.start()
            display_thread.start()

            # Start audio stream
            with sd.InputStream(
                samplerate=self.sample_rate,
                channels=1,
                callback=self.audio_callback,
                blocksize=int(self.sample_rate * 0.1),  # 100ms blocks
                dtype=np.float32
            ):
                print("🎤 Listening... Speak in Malayalam!")

                # Keep running until interrupted
                while self.is_recording:
                    time.sleep(0.1)

        except KeyboardInterrupt:
            print("\n\n⏹️  Stopping transcription...")
        except Exception as e:
            print(f"Error during transcription: {e}")
        finally:
            self.stop_transcription()

    def stop_transcription(self):
        """Stop real-time transcription"""
        self.is_recording = False
        self.is_processing = False
        print("✅ Transcription stopped!")

# Test setup function
def test_setup():
    """Test if everything is set up correctly"""
    try:
        import sounddevice as sd
        import torch
        from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

        print("✅ All required packages imported successfully!")
        print(f"✅ PyTorch version: {torch.__version__}")
        print(f"✅ CUDA available: {torch.cuda.is_available()}")

        # Test audio devices
        print("\n🎤 Available audio devices:")
        devices = sd.query_devices()
        for i, device in enumerate(devices):
            if device['max_input_channels'] > 0:
                print(f"  Input Device {i}: {device['name']}")

        return True
    except ImportError as e:
        print(f"❌ Missing package: {e}")
        return False
    except Exception as e:
        print(f"❌ Setup error: {e}")
        return False

# Main function to run transcription
def main():
    """Main function to run the transcription system"""
    # Configuration
    MODEL_PATH = "/content/drive/MyDrive/wav2vec2-finetuned-final"  # Update this path

    try:
        # Check if model exists
        if not os.path.exists(MODEL_PATH):
            print(f"❌ Model not found at: {MODEL_PATH}")
            print("Please check your model path!")
            return

        print("Initializing transcriber...")
        transcriber = RealTimeMalayalamTranscriber(
            model_path=MODEL_PATH,
            sample_rate=16000,
            chunk_duration=3.0,  # Process 3-second chunks
            overlap=0.5          # 0.5 second overlap
        )

        # Start transcription
        transcriber.start_transcription()

    except Exception as e:
        print(f"❌ Error: {e}")
        print("Make sure your model path is correct and the model files exist!")

# Quick model test function
def test_model(model_path=None):
    """Test if the model can be loaded"""
    if model_path is None:
        model_path = "/content/drive/MyDrive/wav2vec2-finetuned-final"

    try:
        print(f"Testing model at: {model_path}")

        if not os.path.exists(model_path):
            print(f"❌ Model directory not found!")
            return False

        # List files in model directory
        print("Files in model directory:")
        for file in os.listdir(model_path):
            print(f"  - {file}")

        # Try to load model
        processor = Wav2Vec2Processor.from_pretrained(model_path)
        model = Wav2Vec2ForCTC.from_pretrained(model_path)

        print("✅ Model loaded successfully!")
        print(f"✅ Vocabulary size: {len(processor.tokenizer)}")
        return True

    except Exception as e:
        print(f"❌ Error loading model: {e}")
        return False

# Instructions
print("="*60)
print("MALAYALAM REAL-TIME TRANSCRIPTION SYSTEM")
print("="*60)
print()
print("STEP-BY-STEP INSTRUCTIONS:")
print("1. Run: test_setup()      # Test if packages work")
print("2. Run: test_model()      # Test if model loads")
print("3. Run: main()            # Start transcription")
print()
print("Make sure your model is saved at:")
print("/content/drive/MyDrive/wav2vec2-finetuned-final")
print("="*60
      )

test_setup()

In [ ]:
test_model()

In [ ]:
!pip install SpeechRecognition


In [ ]:

import torch
import torchaudio
import librosa
import soundfile as sf
import numpy as np
import gradio as gr
import threading
import queue
import time
import io
from transformers import WhisperForConditionalGeneration, WhisperTokenizer, WhisperProcessor
from datasets import Audio
import speech_recognition as sr
from IPython.display import Audio as IPAudio, display
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries imported successfully!")

In [ ]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"CUDA Version: {torch.version.cuda}")
else:
    print("Using CPU - transcription will be slower")

In [ ]:

class MalayalamTranscriber:
    def __init__(self, model_path_or_name="openai/whisper-small"):
        """
        Initialize the Malayalam transcriber

        Options for model_path_or_name:
        1. "openai/whisper-small" - General multilingual model
        2. "openai/whisper-base" - Faster but less accurate
        3. "openai/whisper-medium" - Better accuracy
        4. "/path/to/your/trained/model" - Your custom trained model
        """
        self.device = device
        self.model_name = model_path_or_name

        print(f"Loading model: {model_path_or_name}")

        try:
            # Load tokenizer and processor
            self.tokenizer = WhisperTokenizer.from_pretrained(model_path_or_name)
            self.processor = WhisperProcessor.from_pretrained(model_path_or_name)

            # Load model
            self.model = WhisperForConditionalGeneration.from_pretrained(
                model_path_or_name,
                torch_dtype=torch.float16 if device.type == "cuda" else torch.float32
            ).to(device)

            print("✅ Model loaded successfully!")

        except Exception as e:
            print(f"Error loading model: {e}")
            print("Falling back to Whisper base model...")
            self.tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-base")
            self.processor = WhisperProcessor.from_pretrained("openai/whisper-base")
            self.model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-base").to(device)

    def transcribe_audio(self, audio_path_or_array, language="ml"):
        """
        Transcribe audio to Malayalam text
        """
        try:
            # Load audio
            if isinstance(audio_path_or_array, str):
                audio, sr = librosa.load(audio_path_or_array, sr=16000)
            else:
                audio = audio_path_or_array
                if len(audio.shape) > 1:
                    audio = audio.mean(axis=1)  # Convert to mono if stereo

            # Process audio
            inputs = self.processor(audio, sampling_rate=16000, return_tensors="pt")
            inputs = inputs.to(self.device)

            # Generate transcription
            with torch.no_grad():
                generated_ids = self.model.generate(
                    inputs.input_features,
                    language=language,
                    task="transcribe",
                    max_length=448,
                    num_beams=5,
                    temperature=0.0
                )

            # Decode transcription
            transcription = self.tokenizer.decode(generated_ids[0], skip_special_tokens=True)
            return transcription

        except Exception as e:
            print(f"Error during transcription: {e}")
            return "Error in transcription"

In [ ]:
import os
# Check current directory
os.listdir('.')
# Check if mounted to Google Drive
os.listdir('/content/drive/MyDrive/')

In [ ]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

# Hugging Face pretrained base model (you can choose multilingual or Indian fine-tuned models too)
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base")
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base")

# Optional: Save to your Drive to avoid redownloading
model.save_pretrained("/content/drive/MyDrive/wav2vec2-base")
processor.save_pretrained("/content/drive/MyDrive/wav2vec2-base")


In [ ]:
# First, try to unmount if already mounted
try:
    from google.colab import drive
    drive.flush_and_unmount()
    # Explicitly remove contents after unmount attempt
    import os
    import shutil
    mount_point = '/content/drive'
    if os.path.exists(mount_point):
        for item in os.listdir(mount_point):
            item_path = os.path.join(mount_point, item)
            if os.path.isdir(item_path):
                shutil.rmtree(item_path)
            else:
                os.remove(item_path)
    print("Unmounted and cleared /content/drive")
except Exception as e:
    print(f"Error during unmount/clear: {e}")
    pass # Ignore errors if it wasn't mounted or already empty

# Clear any cached credentials
import os
os.system('rm -rf ~/.config/Google')
print("Cleared Google auth cache")

# Now try mounting again
from google.colab import drive
print("Attempting to mount Google Drive...")
drive.mount('/content/drive', force_remount=True)
print("Google Drive mount command executed.")

# You can add a check here to see if it was successful
import time
time.sleep(2) # Give it a moment to complete mounting process
if os.path.exists('/content/drive/MyDrive'):
    print("✅ Google Drive successfully mounted!")
else:
    print("❌ Google Drive might not have mounted correctly. Check the output above for errors.")

In [ ]:
import os

# Check what's in your Google Drive
print("Contents of Google Drive:")
drive_contents = os.listdir("/content/drive/MyDrive/")
print(drive_contents)

# Look for your wav2vec2 model folder
model_path = "/content/drive/MyDrive/wav2vec2-finetuned-final"

if os.path.exists(model_path):
    print(f"\nModel folder found at: {model_path}")
    print("Model files:")
    model_files = os.listdir(model_path)
    for file in model_files:
        print(f"  - {file}")
else:
    print("Model folder not found. Let's search for it...")
    for item in drive_contents:
        if "wav2vec2" in item.lower():
            print(f"Found wav2vec2-related folder: {item}")
            model_path = f"/content/drive/MyDrive/{item}"

In [ ]:
import os

model_path = "/content/drive/MyDrive/wav2vec2-finetuned-final"

if os.path.exists(model_path):
    print("📂 Folder exists!")
    print("📄 Contents:")
    print(os.listdir(model_path))
else:
    print("❌ Path does not exist.")


In [ ]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

model_path = "/content/drive/MyDrive/wav2vec2-finetuned-final"  # your local path

# Load your fine-tuned model
print("Loading model...")
# The error suggests the path format is the issue.
# Ensure the path is correct and retry.
# The local_files_only=True flag is crucial for local loading.
try:
    model = Wav2Vec2ForCTC.from_pretrained(model_path, local_files_only=True)
    print("✓ Model loaded successfully!")

    # Load the processor
    try:
        processor = Wav2Vec2Processor.from_pretrained(model_path, local_files_only=True)
        print("✓ Processor loaded successfully!")
    except Exception as e:
        print(f"Processor loading failed: {e}")
        print("You may need to load the processor from the base model")

except Exception as e:
    print(f"Model loading failed: {e}")
    print("Double check that the path to your model folder is correct.")
    print("Ensure that the 'config.json' file and other necessary model files are directly inside this folder:")
    print(model_path)
    # You can add a check here to list files in the directory to confirm
    import os
    if os.path.exists(model_path):
        print("\nFiles in the specified model directory:")
        print(os.listdir(model_path))
    else:
        print("\nThe specified model directory does not exist.")

In [ ]:
print(f"Model type: {type(model)}")
print(f"Model config: {model.config}")
print(f"Model device: {next(model.parameters()).device}")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Move model to GPU if available
model = model.to(device)
print(f"Model is now on: {next(model.parameters()).device}")

In [ ]:
# Try to load the processor that was saved with your model
try:
    processor = Wav2Vec2Processor.from_pretrained(model_path)
    print("✓ Processor loaded from your saved model")
except:
    print("Processor not found in saved model, loading from base model...")
    from transformers import Wav2Vec2Processor
    # You might need to specify the base model you used for training
    processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
    print("✓ Processor loaded from base model")

In [ ]:
# Function to transcribe audio
def transcribe_audio(audio_path):
    import librosa

    # Load audio
    speech, sr = librosa.load(audio_path, sr=16000)

    # Process audio
    inputs = processor(speech, sampling_rate=16000, return_tensors="pt", padding=True)

    # Move inputs to same device as model
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Get predictions
    with torch.no_grad():
        logits = model(**inputs).logits

    # Decode predictions
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids)[0]

    return transcription

print("Model is ready for inference!")
print("You can now use transcribe_audio('path_to_your_audio_file.wav') to transcribe audio")

# Real-time Malayalam Audio Transcription using your fine-tuned Wav2Vec2 model

# Step 1: Install required packages


In [ ]:
!apt update -qq
!apt install portaudio19-dev python3-pyaudio -qq
!pip install pyaudio
!pip install ipywebrtc
!pip install javascript

In [ ]:
!huggingface-cli login


In [ ]:
# Complete Malayalam Audio Transcription System
import torch
import librosa
import numpy as np
import base64
import io
from scipy.io import wavfile
from IPython.display import HTML, display
from google.colab import output
import os

# Install required packages
!pip install pydub transformers datasets

import pydub
from pydub import AudioSegment
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import warnings
warnings.filterwarnings("ignore")

# Initialize model and processor for Malayalam
print("🔄 Loading Malayalam speech recognition model...")

# You can replace this with your specific Malayalam model
# Popular options:
MODEL_NAME = "gvs/wav2vec2-large-xlsr-malayalam"  # Example Malayalam model
# MODEL_NAME = "your-custom-malayalam-model"  # Replace with your model

try:
    # Load processor and model
    processor = Wav2Vec2Processor.from_pretrained(MODEL_NAME)
    model = Wav2Vec2ForCTC.from_pretrained(MODEL_NAME)

    # Set device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.eval()

    print(f"✅ Model loaded successfully on {device}")
    print(f"📱 Model: {MODEL_NAME}")

except Exception as e:
    print(f"❌ Error loading model: {e}")
    print("Please make sure you have the correct model name or path")
    print("\n🔧 Alternative: Load your custom model")
    print("If you have a custom model, replace the MODEL_NAME with:")
    print("- Your model path (e.g., './my-malayalam-model')")
    print("- Your Hugging Face model name")
    print("- Or use a different Malayalam model from Hugging Face")

    # Alternative model loading (uncomment and modify as needed)
    """
    # Example for custom model loading:
    from transformers import AutoProcessor, AutoModelForCTC
    processor = AutoProcessor.from_pretrained("your-model-path")
    model = AutoModelForCTC.from_pretrained("your-model-path")
    """

    raise

# Real-time transcription function
def transcribe_realtime_audio(base64_audio):
    """Process and transcribe real-time audio"""
    try:
        # Decode base64 audio
        audio_data = base64.b64decode(base64_audio)

        # Use pydub to handle WebM format
        audio_io = io.BytesIO(audio_data)

        # Load audio with pydub (handles WebM)
        audio_segment = AudioSegment.from_file(audio_io, format="webm")

        # Convert to mono and set sample rate to 16kHz
        audio_segment = audio_segment.set_channels(1).set_frame_rate(16000)

        # Convert to numpy array
        audio_np = np.array(audio_segment.get_array_of_samples(), dtype=np.float32)

        # Normalize
        audio_float = audio_np / 32768.0

        # Skip if audio is too short
        if len(audio_float) < 8000:  # Less than 0.5 seconds
            return ""

        # Transcribe with your model
        inputs = processor(audio_float, sampling_rate=16000, return_tensors="pt", padding=True)
        inputs = {key: value.to(device) for key, value in inputs.items()}

        with torch.no_grad():
            logits = model(**inputs).logits

        predicted_ids = torch.argmax(logits, dim=-1)
        transcription = processor.batch_decode(predicted_ids)[0].strip()

        # Only return if transcription is meaningful
        if transcription and len(transcription) > 1:
            print(f"🗣️ Malayalam: {transcription}")
            return transcription

        return ""

    except Exception as e:
        print(f"Error in real-time transcription: {e}")
        return ""

# File upload transcription function
def transcribe_uploaded_audio(base64_audio, filename):
    """Process and transcribe uploaded audio file"""
    try:
        # Decode base64 audio
        audio_data = base64.b64decode(base64_audio)

        # Use pydub to handle various audio formats
        audio_io = io.BytesIO(audio_data)

        # Detect format from filename
        file_extension = filename.split('.')[-1].lower()
        format_map = {
            'mp3': 'mp3',
            'wav': 'wav',
            'webm': 'webm',
            'm4a': 'mp4',
            'ogg': 'ogg',
            'flac': 'flac'
        }

        audio_format = format_map.get(file_extension, 'mp3')

        # Load audio with pydub
        audio_segment = AudioSegment.from_file(audio_io, format=audio_format)

        # Convert to mono and set sample rate to 16kHz
        audio_segment = audio_segment.set_channels(1).set_frame_rate(16000)

        # Convert to numpy array
        audio_np = np.array(audio_segment.get_array_of_samples(), dtype=np.float32)

        # Normalize
        audio_float = audio_np / 32768.0

        # Get duration
        duration = len(audio_float) / 16000

        print(f"📁 Processing file: {filename}")
        print(f"⏱️ Duration: {duration:.2f} seconds")
        print(f"🔊 Sample rate: 16kHz")
        print(f"📊 Audio length: {len(audio_float)} samples")

        # Process audio in chunks if it's long (>30 seconds)
        if duration > 30:
            return transcribe_long_audio(audio_float, filename)
        else:
            return transcribe_short_audio(audio_float, filename)

    except Exception as e:
        error_msg = f"❌ Error processing {filename}: {str(e)}"
        print(error_msg)
        return error_msg

def transcribe_short_audio(audio_float, filename):
    """Transcribe short audio files (<=30 seconds)"""
    try:
        # Transcribe with your model
        inputs = processor(audio_float, sampling_rate=16000, return_tensors="pt", padding=True)
        inputs = {key: value.to(device) for key, value in inputs.items()}

        with torch.no_grad():
            logits = model(**inputs).logits

        predicted_ids = torch.argmax(logits, dim=-1)
        transcription = processor.batch_decode(predicted_ids)[0].strip()

        if transcription and len(transcription) > 1:
            result = f"✅ Transcription for {filename}:\n🗣️ Malayalam: {transcription}"
            print(result)
            return result
        else:
            result = f"⚠️ No clear speech detected in {filename}"
            print(result)
            return result

    except Exception as e:
        error_msg = f"❌ Error transcribing {filename}: {str(e)}"
        print(error_msg)
        return error_msg

def transcribe_long_audio(audio_float, filename):
    """Transcribe long audio files (>30 seconds) in chunks"""
    try:
        chunk_length = 30 * 16000  # 30 seconds in samples
        chunks = []
        transcriptions = []

        # Split audio into chunks
        for i in range(0, len(audio_float), chunk_length):
            chunk = audio_float[i:i + chunk_length]
            if len(chunk) > 8000:  # At least 0.5 seconds
                chunks.append(chunk)

        print(f"🔄 Processing {len(chunks)} chunks for {filename}...")

        # Process each chunk
        for i, chunk in enumerate(chunks):
            try:
                inputs = processor(chunk, sampling_rate=16000, return_tensors="pt", padding=True)
                inputs = {key: value.to(device) for key, value in inputs.items()}

                with torch.no_grad():
                    logits = model(**inputs).logits

                predicted_ids = torch.argmax(logits, dim=-1)
                transcription = processor.batch_decode(predicted_ids)[0].strip()

                if transcription and len(transcription) > 1:
                    transcriptions.append(f"[{i+1:02d}] {transcription}")
                    print(f"📝 Chunk {i+1}/{len(chunks)}: {transcription}")

            except Exception as e:
                print(f"⚠️ Error processing chunk {i+1}: {e}")
                continue

        if transcriptions:
            full_transcription = "\n".join(transcriptions)
            result = f"✅ Complete transcription for {filename}:\n{full_transcription}"
            return result
        else:
            result = f"⚠️ No clear speech detected in {filename}"
            return result

    except Exception as e:
        error_msg = f"❌ Error processing long audio {filename}: {str(e)}"
        print(error_msg)
        return error_msg

# Register callbacks for JavaScript
output.register_callback('transcribe_audio', transcribe_realtime_audio)
output.register_callback('transcribe_file', transcribe_uploaded_audio)

# Complete interface with both real-time and file upload
complete_interface = HTML("""
<div style="max-width: 800px; margin: 0 auto; font-family: Arial, sans-serif;">
    <h2 style="text-align: center; color: #2E7D32;">🎤 Malayalam Audio Transcription System</h2>

    <!-- Real-time Recording Section -->
    <div style="margin: 20px 0; padding: 20px; border: 2px solid #4CAF50; border-radius: 10px; background-color: #f9f9f9;">
        <h3 style="color: #2E7D32; margin-top: 0;">🔴 Real-time Recording</h3>
        <button id="startBtn" onclick="startRealTimeRecording()" style="background-color: #4CAF50; color: white; padding: 10px 20px; border: none; border-radius: 5px; cursor: pointer; margin: 5px;">
            Start Real-time Recording
        </button>
        <button id="stopBtn" onclick="stopRealTimeRecording()" disabled style="background-color: #f44336; color: white; padding: 10px 20px; border: none; border-radius: 5px; cursor: pointer; margin: 5px;">
            Stop Recording
        </button>
        <div id="realtime-status" style="font-size: 16px; font-weight: bold; margin: 10px 0;">Ready to record Malayalam audio...</div>
        <div id="realtime-transcription" style="margin-top: 10px; padding: 10px; background-color: white; border: 1px solid #ddd; border-radius: 5px; min-height: 80px; max-height: 200px; overflow-y: auto;"></div>
    </div>

    <!-- File Upload Section -->
    <div style="margin: 20px 0; padding: 20px; border: 2px solid #2196F3; border-radius: 10px; background-color: #f3f8ff;">
        <h3 style="color: #1976D2; margin-top: 0;">📁 Audio File Upload</h3>
        <div style="margin: 15px 0;">
            <input type="file" id="audioFileInput" accept="audio/*" style="margin: 10px 0; padding: 5px;">
            <button id="uploadBtn" onclick="uploadAndTranscribe()" style="background-color: #2196F3; color: white; padding: 10px 20px; border: none; border-radius: 5px; cursor: pointer; margin-left: 10px;">
                Upload & Transcribe
            </button>
        </div>
        <div style="font-size: 14px; color: #666;">
            Supported formats: MP3, WAV, M4A, OGG, FLAC, WebM (Max size: 100MB)
        </div>
        <div id="upload-status" style="font-size: 16px; font-weight: bold; margin: 10px 0;">Select an audio file to transcribe...</div>
        <div id="upload-transcription" style="margin-top: 10px; padding: 10px; background-color: white; border: 1px solid #ddd; border-radius: 5px; min-height: 80px; max-height: 300px; overflow-y: auto; white-space: pre-wrap;"></div>
    </div>

    <!-- Instructions -->
    <div style="margin: 20px 0; padding: 15px; background-color: #fff3e0; border-radius: 5px; border-left: 4px solid #ff9800;">
        <h4 style="color: #ef6c00; margin-top: 0;">📋 Instructions:</h4>
        <ul style="color: #bf360c;">
            <li><strong>Real-time:</strong> Click "Start Recording" and speak in Malayalam</li>
            <li><strong>File Upload:</strong> Select an audio file and click "Upload & Transcribe"</li>
            <li>Long files (>30 seconds) will be processed in chunks</li>
            <li>Ensure clear audio quality for better transcription</li>
        </ul>
    </div>
</div>

<script>
// Real-time recording variables
let mediaRecorder;
let isRecording = false;
let audioChunks = [];
let recordingInterval;

// Real-time recording functions
async function startRealTimeRecording() {
    try {
        const stream = await navigator.mediaDevices.getUserMedia({
            audio: {
                sampleRate: 16000,
                channelCount: 1,
                echoCancellation: true,
                noiseSuppression: true
            }
        });

        mediaRecorder = new MediaRecorder(stream, {
            mimeType: 'audio/webm;codecs=opus'
        });

        isRecording = true;
        document.getElementById('startBtn').disabled = true;
        document.getElementById('stopBtn').disabled = false;
        document.getElementById('realtime-status').innerHTML = '🔴 Recording... Speak in Malayalam!';
        document.getElementById('realtime-transcription').innerHTML = '';

        // Process audio in chunks every 3 seconds
        recordingInterval = setInterval(() => {
            if (isRecording && mediaRecorder.state === 'recording') {
                mediaRecorder.stop();
                setTimeout(() => {
                    if (isRecording) {
                        mediaRecorder.start();
                    }
                }, 100);
            }
        }, 3000);

        mediaRecorder.ondataavailable = function(event) {
            if (event.data.size > 0) {
                audioChunks.push(event.data);
                processRealtimeAudioChunk(event.data);
            }
        };

        mediaRecorder.start();

    } catch (err) {
        console.error('Error accessing microphone:', err);
        document.getElementById('realtime-status').innerHTML = '❌ Error: Cannot access microphone';
    }
}

function stopRealTimeRecording() {
    isRecording = false;

    if (mediaRecorder && mediaRecorder.state !== 'inactive') {
        mediaRecorder.stop();
        mediaRecorder.stream.getTracks().forEach(track => track.stop());
    }

    if (recordingInterval) {
        clearInterval(recordingInterval);
    }

    document.getElementById('startBtn').disabled = false;
    document.getElementById('stopBtn').disabled = true;
    document.getElementById('realtime-status').innerHTML = '⏹️ Recording stopped';
}

async function processRealtimeAudioChunk(audioBlob) {
    try {
        const reader = new FileReader();
        reader.onload = function() {
            const base64Audio = reader.result.split(',')[1];
            google.colab.kernel.invokeFunction('transcribe_audio', [base64Audio], {})
                .then(result => {
                    if (result.data && result.data.trim()) {
                        const transcriptionDiv = document.getElementById('realtime-transcription');
                        transcriptionDiv.innerHTML += result.data + '<br>';
                        transcriptionDiv.scrollTop = transcriptionDiv.scrollHeight;
                    }
                });
        };
        reader.readAsDataURL(audioBlob);
    } catch (error) {
        console.error('Error processing real-time audio chunk:', error);
    }
}

// File upload functions
async function uploadAndTranscribe() {
    const fileInput = document.getElementById('audioFileInput');
    const file = fileInput.files[0];

    if (!file) {
        document.getElementById('upload-status').innerHTML = '⚠️ Please select an audio file first';
        return;
    }

    // Validate file size (100MB max)
    if (file.size > 100 * 1024 * 1024) {
        document.getElementById('upload-status').innerHTML = '❌ File too large. Maximum size is 100MB';
        return;
    }

    // Validate file type
    const allowedTypes = ['audio/mpeg', 'audio/wav', 'audio/mp4', 'audio/ogg', 'audio/webm', 'audio/flac', 'audio/m4a'];
    if (!allowedTypes.some(type => file.type.includes('audio'))) {
        document.getElementById('upload-status').innerHTML = '❌ Please select a valid audio file';
        return;
    }

    document.getElementById('upload-status').innerHTML = '🔄 Processing audio file... Please wait';
    document.getElementById('upload-transcription').innerHTML = 'Processing...';
    document.getElementById('uploadBtn').disabled = true;

    try {
        const reader = new FileReader();
        reader.onload = function() {
            const base64Audio = reader.result.split(',')[1];
            google.colab.kernel.invokeFunction('transcribe_file', [base64Audio, file.name], {})
                .then(result => {
                    document.getElementById('upload-status').innerHTML = '✅ Transcription completed';
                    document.getElementById('upload-transcription').innerHTML = result.data || 'No transcription result';
                    document.getElementById('uploadBtn').disabled = false;
                })
                .catch(error => {
                    console.error('Transcription error:', error);
                    document.getElementById('upload-status').innerHTML = '❌ Error during transcription';
                    document.getElementById('upload-transcription').innerHTML = 'Transcription failed. Please try again.';
                    document.getElementById('uploadBtn').disabled = false;
                });
        };
        reader.readAsDataURL(file);
    } catch (error) {
        console.error('File reading error:', error);
        document.getElementById('upload-status').innerHTML = '❌ Error reading file';
        document.getElementById('uploadBtn').disabled = false;
    }
}

// File input change handler
document.addEventListener('DOMContentLoaded', function() {
    const fileInput = document.getElementById('audioFileInput');
    if (fileInput) {
        fileInput.addEventListener('change', function() {
            const file = this.files[0];
            if (file) {
                const fileSize = (file.size / (1024 * 1024)).toFixed(2);
                document.getElementById('upload-status').innerHTML =
                    `📁 Selected: ${file.name} (${fileSize} MB) - Ready to transcribe`;
            }
        });
    }
});
</script>
""")

# Display the complete interface
print("🚀 Complete Malayalam Speech Recognition System")
print("=" * 60)
print("🔴 Real-time: Start recording and speak in Malayalam")
print("📁 File Upload: Select and upload audio files for transcription")
print("🔄 Long files will be processed in chunks automatically")
print("=" * 60)
display(complete_interface)